<img align="right" src="files/logo_grupo_en_verde.png" style="width: 200px;"/> 
<br>
<br>
<br>
<br>
<img align="left" src="files/ee.png" style="width: 800px;"/>

# Instalação e Configuração da API Python

In [ ]:
!cat files/eeInstall.sh

## Verificando a Instalação

In [ ]:
# Importando e Inicializando a API do Google Engine
import ee
ee.Initialize()

In [ ]:
# Importando dados da plataforma
srtm = ee.Image("CGIAR/SRTM90_V4")
info = srtm.getInfo()
print(info)

In [ ]:
list(info.keys())

# Images

## Visualização de dados utilizando Ipython.display

In [ ]:
from IPython.display import Image
Image(url=srtm.getThumbUrl())

In [ ]:
# Adicionando parâmetros para melhorar a visualização
Image(url=srtm.getThumbUrl({'min':0, 'max':3000}))

Por padrão o método **.getTumbUrl()** retorna o mapa com toda a sua extensão (neste caso, global). É possível especificar parâmetros para uma extensão menor.

## Definindo uma região para a plotagem da base de dados

In [ ]:
# Definindo um ponto para a cidade de Viçosa - MG
pointVico = ee.Geometry.Point(-42.8786, -20.7549)

# Criando uma região de buffer em  torno do ponto criado (m)
region_vicosa = pointVico.buffer(50000).bounds().getInfo()['coordinates']

# Plotando os dados
Image(url=srtm.getThumbUrl({'min':0, 'max':1000, 'region':region_vicosa}))

#  Image Collection

## Carregando e Filtrando Image Collections

In [ ]:
# Define a collection
l8 = ee.ImageCollection('LANDSAT/LC8_L1T_TOA')

# Filtra a collection para o período ('2016-05-01', '2017-06-30')
filtered = l8.filterDate('2016-05-01', '2017-06-30');

# O redutor de mosaico seleciona o pixel mais recente nas áreas de sobreposição
l8_image = filtered.mosaic()

# Define uma região que cobre a cidade de viçosa - MG
point = ee.Geometry.Point(-42.8786, -20.7549)
region_vicosa = point.buffer(50000).bounds().getInfo()['coordinates']

# Plota a collection
Image(url=l8_image.getThumbUrl({'region':region_vicosa}))

É possível obter todas as informações da imagem através de seus metadados

In [ ]:
# Carregando imagem
image = ee.Image('LANDSAT/LC8_L1T/LC80440342014077LGN00')
Image(url=image.getThumbUrl())

In [ ]:
# Retorna lista com informação sobre a imagem
imageInfo = image.getInfo()
imageInfo

In [ ]:
imageInfo.keys()

In [ ]:
# Exibindo as bandas da imagem
for i in range(len(imageInfo['bands'])):
    print(imageInfo['bands'][i]['id'])

In [ ]:
imageInfo['properties']

In [ ]:
imageInfo['properties']['CLOUD_COVER']

## Correção Atmosférica (L8)

$L_{\lambda} = M_{L}Q_{cal} + A_{L}$

$L_{\lambda} =$ TOA spectral radiance $W/(m^{2}*srad*\mu m)$ <br>
$M_{L} = $ Band-specific multiplicative rescaling factor from the metadata (RADIANCE_MULT_BAND_x, where x is the band number)<br>
$Q_{cal} = $ Band-specific additive rescaling factor from the metadata (RADIANCE_ADD_BAND_x, where x is the band number)<br>
$A_{L} = $ Quantized and calibrated standard product pixel values (DN)<br>


> **Radiância:** Densidade do fluxo luminoso em elemento de superfície de uma fonte de energia.

In [ ]:
# Obtendo parâmetros da imagem
ML = imageInfo['properties']['RADIANCE_MULT_BAND_6']
Qcal = imageInfo['properties']['RADIANCE_ADD_BAND_6']

Al = image.select('B6')
print("ML = %f" %(ML))
print("Qcal = %f" %(Qcal))

In [ ]:
Image(url=Al.getThumbUrl())

In [ ]:
# Calculando a correção na plataforma Earth Engine
L = image.expression(
    'ML * Qcal * Al',{
        'ML': ML,
        'Qcal': Qcal,
        'Al': Al
    }
)

In [ ]:
Image(url=L.getThumbUrl())

## Correção para o Topo da Atmosfera (TOA)

$\rho\lambda' = M_{\rho}Q_{cal} + A_{\rho}$

$\rho\lambda' =$ TOA planetary reflectance, without correction for solar angle.  Note that $\rho\lambda'$ does not contain a correction for the sun angle<br>
$M_{\rho} = $ Band-specific multiplicative rescaling factor from the metadata (REFLECTANCE_MULT_BAND_x, where x is the band number)<br>
$Q_{cal} = $ Quantized and calibrated standard product pixel values (DN)<br>
$A_{L} = $ Band-specific additive rescaling factor from the metadata (REFLECTANCE_ADD_BAND_x, where x is the band number)<br>

> **Reflecância:** $R = \frac{F_{r}}{F_{T}}$ <br>
> $F_{r} = $ Fluxo de radiação eletromagnética refletido. <br>
> $F_{t} = $ Fluxo de radiação eletromagnética incidente.

> Como exercício façam a correção em função do Ângulo Solar <br>
> Mais informações: https://landsat.usgs.gov/using-usgs-landsat-8-product

## Trabalhando com Bandas (Plots)

Composição de bandas podem ser feitas para melhorar a visualização dos dados 

In [ ]:
# Plotando imagem com composição de bandas para obter uma melhor visualização
# l8_image é a imagem filtrada e reduzida com pixels mais recentes
Image(url=l8_image.getThumbUrl({
    'region':region_vicosa,
    'bands':'B4,B3,B2',
    'min':0,
    'max':0.3
}))

## Reducing Image Collections

Reducers são métodos para agregar dados ao longo do tempo, espaço, bandas, arrays e outras estruturas de dados no Earth Engine.

### ImageCollection.mosaic Reducer

In [ ]:
# Pixel mais recente
filtDate = l8.filterDate('2016-01-01', '2017-01-01')

l8_vico = filtDate.mosaic()
Image(url=l8_vico.getThumbUrl({
    'region':region_vicosa,
    'bands':'B4,B3,B2',
    'min':0,
    'max':0.3
}))

### ImageCollection.median Reducer

In [ ]:
# mediana dos pixels
l8_vico = filtDate.median()
Image(url=l8_vico.getThumbUrl({
    'region':region_vicosa,
    'bands':'B4,B3,B2',
    'min':0,
    'max':0.3
}))

### ImageCollection.min Reducer

In [ ]:
# Pixel de valor mínimo
l8_vico = filtDate.min()
Image(url=l8_vico.getThumbUrl({
    'region':region_vicosa,
    'bands':'B4,B3,B2',
    'min':0,
    'max':0.3
}))